In [1]:
import numpy as np


def load_data():
    with open('continious_value_data.txt') as f:
        lines = f.readlines()

    x = np.empty((len(lines), 8), dtype=float)
    y = np.empty(len(lines), dtype=float)

    for i in range(len(lines)):
        line = lines[i].strip().split(',')
        x[i] = line[:8]
        y[i] = line[8]

    return x, y


x, y = load_data()
x, y

(array([[0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.697, 0.46 ],
        [1.   , 0.   , 1.   , 0.   , 0.   , 0.   , 0.774, 0.376],
        [1.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.634, 0.264],
        [0.   , 1.   , 0.   , 0.   , 1.   , 1.   , 0.403, 0.237],
        [1.   , 1.   , 0.   , 1.   , 1.   , 1.   , 0.481, 0.149],
        [0.   , 2.   , 2.   , 0.   , 2.   , 1.   , 0.243, 0.267],
        [2.   , 1.   , 1.   , 1.   , 0.   , 0.   , 0.657, 0.198],
        [1.   , 1.   , 0.   , 0.   , 1.   , 1.   , 0.36 , 0.37 ],
        [2.   , 0.   , 0.   , 2.   , 2.   , 0.   , 0.593, 0.042],
        [0.   , 0.   , 1.   , 1.   , 1.   , 0.   , 0.719, 0.103],
        [0.   , 0.   , 1.   , 0.   , 0.   , 0.   , 0.608, 0.318],
        [2.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.556, 0.215],
        [1.   , 1.   , 0.   , 0.   , 1.   , 0.   , 0.437, 0.211],
        [1.   , 1.   , 1.   , 1.   , 1.   , 0.   , 0.666, 0.091],
        [2.   , 2.   , 2.   , 2.   , 2.   , 0.   , 0.245, 0.057],
        [2

In [2]:
N, M = x.shape

In [13]:
# 给定y和某列值的情况下，求离散概率
# p(x|y, col_val)
def get_discrete_prob(col, value, _y):
    _x = x[y == _y]
    # +1 做拉普拉斯平滑
    fenzi = (_x[:, col] == value).sum() + 1
    # +col列取值数量，做拉普拉斯平滑
    fenmu = len(_x) + len(np.unique(_x[:, col]))
     
    return fenzi / fenmu

get_prob(0, 0, 0)

0.36363636363636365

In [14]:
# 给定y和某列值的情况下，求连续概率
# p(x|y, col_val)
# 用正太分布概率密度函数
def get_continuous_prob(col, value, _y):
    sqrt_2_pi = (2 * np.pi)**0.5
    _x = x[y == _y]
    
    # 求均值和方差
    mean = _x[:, col].mean()
    sigma = _x[:, col].std()
    
    p = 1 / (sqrt_2_pi * sigma)
    
    fenzi = (value - mean)**2
    fenmu = sigma**2 * 2
    
    return p* np.exp(-fenzi / fenmu)

In [15]:
get_continuous_prob(6, 0.697, 0)

1.9624922010858166

In [16]:
def predict(_x):
    # y有两种取值：0，1
    # 所以会预测出两个概率
    ps = np.zeros(2)
    
    for _y in range(2):
        for col in range(M):
            p = None
            if col == 6 or col == 7:
                p = get_continuous_prob(col, _x[col], _y)
            else:
                p = get_discrete_prob(col, _x[col], _y)
            
            # 取对数，连乘变连加
            ps[_y] += np.log(p)
            
    return ps.argmax()

predict(x[0])

0

In [17]:
correct = 0
for xi, yi in zip(x, y):
    if predict(xi) == yi:
        correct += 1
correct / N

0.8235294117647058